# Forward Step Wise Variable Selection for Dummies

In [47]:
# Import status
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
import numpy as np

In [48]:
# Read in basetable_ex2_4
df = pd.read_csv("/Users/christine/VSCode/MBAN 6110 - remote/schulich_MBAN6110/forward_stepwise_example/basetable_ex2_4.csv")

In [49]:
# Check dataframe
df.head()

,target,gender_F,income_high,income_low,country_USA,country_India,country_UK,age,time_since_last_gift,time_since_first_gift,max_gift,min_gift,mean_gift,number_gift
0,0,1,0,1,0,1,0,65,530,2265,166.0,87.0,116.00,7
1,0,1,0,0,0,1,0,71,715,715,90.0,90.0,90.00,1
2,0,1,0,0,0,1,0,28,150,1806,125.0,74.0,96.00,9
3,0,1,0,1,1,0,0,52,725,2274,117.0,97.0,104.25,4
4,0,1,1,0,1,0,0,82,805,805,80.0,80.0,80.00,1


In [50]:
# Algorithm for AUC and forward pass step forward model
# Assume you understand the logic already.

# Requirements:
# A function to calculate AUC
# A function to return the "next_best", or the highest AUC. 
# A loop to go thru this, and remove that next_best

In [51]:
# Define the AUC score function
# Takes in the data source table - basetable, variables and target

def auc(variables, target, basetable):

    # Define the variables for initialization
    X = basetable[variables]
    y = basetable [target]

    # Train the logistic regression model
    logreg = linear_model.LogisticRegression()

    # Fit the model. Has been trained
    logreg.fit(X,y)

    # Create predictions from X. Technically there should be a train-test split, but whatever...
    # We take the entire basetable variables, which is X, and then we take all of whatever is in 2nd Column, or "1"
    # Why do we do that --> predictions give an array of results (A,B), where A is a num if Target = 0 
    # B where num if Target = 1 - what we are looking for!! 

    predictions = logreg.predict_proba(X)[:,1]

    # Calculate AUC score. Need true y and the predictions
    auc_score = roc_auc_score(y, predictions)

    return auc_score

In [52]:
# Use the AUC score function
# note that df is the "base table".
# Our AUC function works!

call = auc(["gender_F","country_USA"], ["target"], df)
call

/Users/christine/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.519594279083611

In [53]:
# Calculate Next Best function.
# This function is critical, it will loop thru and select the best one

def next_best(current_variables, candidate_variables, target, basetable):

    # Set up.
    best_auc = -1 # Pick an initialization lower than 0. Otherwise the evaluation step will not work.
    best_variable = None

    # Loop thru the candidate variables given the current ones..
    # Remember, current variables tracks whatever is in the "set" of model variables selected for the model. Thru the logic.

    # The evaluation, one by one:
    for v in candidate_variables:
        auc_v = auc(current_variables + [v], target, basetable)
        
        # The most important step - check if AUC is better or higher.
        # This will only update if it's actually better...
        if auc_v > best_auc:
            best_auc = auc_v  # Update the best AUC
            best_variable  = v
        
    return best_variable

In [54]:
# Test it works
current_variables = ["age", "gender_F"] # They passed the test, are already in the model set
candidate_variables = ["min_gift", "max_gift", "mean_gift"]

next_variable = next_best(current_variables, candidate_variables, ["target"], df)
next_variable


/Users/christine/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/christine/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/christine/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'max_gift'

In [55]:
# Create the looping process now

current_variables = [] # They passed the test, are already in the model set
candidate_variables = ["min_gift",  "max_gift", "mean_gift", "age", "gender_F", "country_USA", "income_low"]

# Limit the total iterations:
max_iterations = 5
num_iterations = min(max_iterations, len(candidate_variables))

# Looping process
for i in range (0,num_iterations):
    # Pick the next_best from a list of the candidate variables
    next_var_selection = next_best(current_variables, candidate_variables, ["target"], df)

    # Update and add the best one to the current_variables list
    current_variables.append(next_var_selection)

    # Remove from candidate list and continue the process
    candidate_variables.remove(next_var_selection)


/Users/christine/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/christine/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/christine/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/christine/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was e

In [58]:
# Test that it works
call = auc(current_variables, ["target"], df)
print(call)

0.7155429426932984


/Users/christine/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [59]:
# Important step is to check correlations...
correlation = np.corrcoef(df["min_gift"], df["mean_gift"])[0,1]
print(round(correlation,2))

# Just like how df.corr() checks Pearson-R for linear correlation, there is also np.corrcoef


0.76
